<a href="https://colab.research.google.com/github/emlingo/Repository1/blob/master/PRP_Function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages and libraries

import numpy as np
from sklearn import linear_model
from numpy import linalg as LA


In [ ]:
def PRP(A,b,p, maxiter, epsilon):
  m,n = np.shape(A)

  b_av = b/p
  R_hat = np.hstack([b_av]*p)

  X_hat = np.zeros((n,1))
  omega = np.ones((p,1))/p

  tol = 0
  iter = 0

  while tol == 0:

    R = omega[i]*R_hat.sum(axis=0)

    # done in parallel (eventually). Will start with a normal for loop.
    a = 0
    delta_x = np.zeros((n,p))

    for i in range(p):
      
      # how will we determine the regularization param?
      # I have hard coded it to be 1e-3 for now.
      model = linear_model.lasso(alpha = 1e-3)
      sol = model.fit(A[:,a:a+n/p], R[:,i])

      delta_x[:,i] = sol

      # recalculate residual
      R_hat[:,i] = R[:,i] - A[:,a:a+n/p]*sol

      a = a + n/p

    # end of (parallel) for
    
    # update solution for X_hat
    delta_x_tall = delta_x.flatten('F')
    X_hat = X_hat + delta_x_tall


    # calculate difference from previous iteration
    diff = LA.norm(delta_x_tall, 2)
    
    iter += 1

    # determines whether stop criteria is met  
    if diff < epsilon:
      tol = 1

    if iter > maxiter:
      tol = 1

  # end outer loop

  return X_hat, R_hat.sum(axis=0)


In [ ]:
# generate synthetic (sparse) data

n=500
d =1000
k=100
stdev=1

A = np.random.normal(0,1,[n,d])       # matrix of features

eps = np.random.normal(0,stdev,[n,1]) # noise

x = np.zeros([d,1])                   # coefficients vector

for i in range(k):                    # set nonzero values of w
  x[i]=(i+1)/k

b = np.dot(A,x)+eps                   # vector of labels



In [ ]:
# check sparsity of system

print('Nonzero entries in A: {} of {}'.format(np.count_nonzero(A),n*d))
print('Nonzero entries in x: {} of {}'.format(np.count_nonzero(x),d))
print('Nonzero entries in b: {} of {}'.format(np.count_nonzero(b),n))

Nonzero entries in A: 500000 of 500000
Nonzero entries in x: 100 of 1000
Nonzero entries in b: 500 of 500


In [ ]:
m = np.matrix([[1,2], [3,4]])
print(m[:,0])

m1 = m.flatten()
print(m1)

m2 = m.flatten('F')
print(m2)

[[1]
 [3]]
[[1 2 3 4]]
[[1 3 2 4]]
